# Прогнозирование стоимости автомобиля по характеристикам

### Описание признаков в тестовом датасете:

bodyType - тип кузова,
brand - марка автомобиля,
car_url - адрес страницы с объявлением,
color - цвет автомобиля,
complectation_dict - словарь с параметрами комплектации авто,
description - описание автомобиля,
engineDisplacement - объем двигателя,
enginePower - мощность двигателя,
equipment_dict - словарь оснащения авто,
fuelType - тип топлива,
image - изображение,
mileage - пробег,
modelDate - дата выпуска модели,
model_info - информация о модели,
model_name - название модели,
name - имя,
numberOfDoors - кол-во дверей,
parsing_unixtime - когда был произведен парсинг данных,
priceCurrency - валюта,
productionDate - год производства,
sell_id,
super_gen - словарь с дополинтельной информацией,
vehicleConfiguration - конфигурация транспортного средства,
vehicleTransmission - трансмиссия,
vendor - регион изготовления,
Владельцы - кол-во владельцев,
Владение - период владения,
ПТС,
Привод,
Руль,
Состояние,
Таможня

## Парсинг.
В результате парсинга данных результат оказался неудовлетворительным. С учетом отсутствия достаточного количества времени принял решение воспользоваться набором данных с Kaggle.

In [ ]:
# использовалась при работе в Colabe Notebooks
# used when working at Colabe Notebooks
#from google.colab import drive

## Импорт библиотек

In [ ]:
import numpy as np
import pandas as pd
import sys
import re
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import time
import xgboost as xgb

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from pandas_profiling import ProfileReport
from scipy.stats import ttest_ind
from itertools import combinations
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
import xgboost as xgb
from sklearn.base import clone

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, RobustScaler, MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import f_regression, mutual_info_regression
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, BaggingRegressor
from sklearn.ensemble import StackingRegressor

%matplotlib inline
warnings.simplefilter('ignore')
sns.set()

In [ ]:
# использовалась при работе в Colabe Notebooks
# used when working at Colabe Notebooks
#drive.mount('/drive')

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [ ]:
# зафиксируем RANDOM_SEED, чтобы эксперименты были воспроизводимы:
RANDOM_SEED = 42

## Пользовательские функции.

In [ ]:
def mape(y_true, y_pred):
    """
    Функция возвращает метрику MAPE.
    На вход получает массив реальных и предсказанных значений y.
    """
    return np.mean(np.abs((y_pred-y_true)/y_true))

In [ ]:
def months_to_sent(months):
    """
    Функция, которая принимает на вход количество месяцев, а возвращает строку в формате "x  месяц(-а -ев)"
    """
    if months == 1:
        return f'{months} месяц'
    elif 2 <= months <= 4:
        return f'{months} месяца'
    return f'{months} месяцев'


def years_to_sent(years):
    """
    Функция, которая принимает на вход количество лет, а возвращает строку в формате "x  лет/год/года"
    """
    if 11 <= years <= 14 or 5 <= years % 10 <= 9 or years % 10 == 0:
        return f'{years} лет'
    elif years % 10 == 1:
        return f'{years} год'
    elif 2 <= years % 10 <= 4:
        return f'{years} годa'


def tenure(row):
    """
    Функция, которая приводит содержимое ячейки "Владение" в train к тому же виду, как в test.
    Вход: строка series (строка колонки).
    Выход: возвращает вместо словаря информацию в виде "x лет y месяцев"
    """
    row = re.findall('\d+', str(row))  # находим все целые числа в строке
    if row != []:
        years = 2020 - (int(row[0])+1)  # 2020, т.к. данные собраны в 2020 году
        months = 2 + (12 - int(row[1]))
        if years < 0:
            return months_to_sent(int(row[1]))
        elif years == 0 and months < 12:
            return months_to_sent(months)
        elif years >= 0 and months == 12:
            return years_to_sent(years + 1)
        elif years >= 0 and months > 12:
            return years_to_sent(years + 1)+' и '+months_to_sent(months - 12)
        elif years > 0 and months < 12:
            return years_to_sent(years)+' и '+months_to_sent(months)
        return None

In [ ]:
def convert_engineDisplacement_to_float(row):
    """
    Функция, которая принимает строку, содержащую информацию про объеме двигателя, находит числовое значение (через точку, например 2.0),
    возвращает значение литров во float формате.
    """
    row = str(row)
    volume = re.findall('\d\.\d', row)
    if volume == []:
        return None
    return float(volume[0])

In [ ]:
def intitial_eda_checks(df, missing_percent):
    '''
    Функция принимает на вход датафрейм, а также заданный порог % пустых значений, который хотим обработать. 
    На выход выводит на экран информацию о сумме пустых значений для всех колонок, а также проце
    '''
    if df.isnull().sum().sum() > 0:
        mask_total = df.isnull().sum().sort_values(ascending=False)
        total = mask_total[mask_total > 0]

        mask_percent = df.isnull().mean().sort_values(ascending=False)
        percent = mask_percent[mask_percent > 0]

        series = mask_percent[mask_percent > missing_percent]
        columns = series.index.to_list()

        missing_data = pd.DataFrame(pd.concat(
            [total, round(percent*100, 2)], axis=1, keys=['Количество', '%']))
        print('Сумма и процент значений NaN:\n \n')
        display(missing_data)
    else:
        print('NaN значения не найдены.')

In [ ]:
def num_of_months(row):
    """
    Функция перевеодит формат данных "x лет y месяца(ев)" в количество месяцев.
    
    Вход: строка series.
    Выход: количество месяцев.
    """
    if pd.notnull(row) and row != 'no_data':
        list_ownership = row.split()  # разделяем строку на список из месяца и лет
        if len(list_ownership) == 2:  # если содержится информация только про годы
            if list_ownership[1] in ['год', 'года', 'лет']:
                return int(list_ownership[0])*12
            return int(list_ownership[0])  # переводим год в месяцы
        # случай, когда содержится инфо только про месяц
        # прибавляем к годам*12 количетсво месяцев
        return int(list_ownership[0])*12 + int(list_ownership[3])

In [ ]:
def get_boxplot(column):
    """
    Функция для отрисовки коробочной диаграммы для нечисловых признаков.
    
    На вход получаем колоноку, для которой строим график.
    График отрисовываем относительно целевой переменной pricing, ограниченной по квантилю.
    """
    fig, ax = plt.subplots(figsize=(25, 5))
    plt.subplots_adjust(wspace=0.5)
    sns.boxplot(x=column, y='price',
                data=data[data['train'] == 1],
                ax=ax)
    plt.xticks(rotation=45)
    # поскольку в price много выбросов, огриничимся 75% квантилем
    ax.set_ylim(0, (data.price.quantile(0.75) + 8 *
                    (data.price.quantile(0.75) - data.price.quantile(0.25))))
    ax.set_title('Boxplot для ' + column)
    plt.show()

In [ ]:
def analyze_cat_cols(col):
    """
    Функция для анализа категориального признака.
    
    На вход получаем колонку, для которой проводим анализ.
    
    В результате выводи графики:
        1. Распределение количества объявлений по признаку.
            Дополнительно, выводится график (прямая) возможного среднего количетсва объявлений по всем категориям признака, чтобы сравнить, насколько
            отклоняется от среднего та или иная категория признака.
        2. График боксплот для признака/целевой переменной (цены).
    """
    fig, axes = plt.subplots(figsize=(25, 5))
    # Посчитаем среднее значение количества объявлений по всем категориям признака
    mean = data[col].value_counts().values.mean()
    x = data[col].unique()
    # Построим графики по признаку и для среднего
    sns.histplot(data=data, x=data[col],
                 stat='count', bins=data[col].nunique())
    axes.plot(x, [mean for i in x], '--', color='r')
    plt.xticks(rotation=45)
    plt.title('Распределение количества объясвлений по '+col)
    plt.show()
    
    # Строим боксплот
    get_boxplot(col)

In [ ]:
def km_per_year(row):
    """
    Функция делает пересчет км пробега автомобиля в год. 
    Для расчета количества лет использование берется год производства автомобиля.
    
    Вход: строка датафрейма.
    Выход: информация о км/год.
    """
    if row['mileage'] != 0:
        # обрабатываем случаи, когда мошина произведена в год сбора данных, чтоб не делить на 0
        if row['parsing_date'].year - row['productionDate'] == 0:
            return row['mileage']
        else:
            return row['mileage']/(row['parsing_date'].year - row['productionDate'])
    return 0  # возвращаем 0, если машина без пробега (новая)

In [ ]:
def prod_date_range(row, year_lim):
    """
    Функция для создания столбца с информацией, старше ли авто, чем year_lim.
    Вход: строка, установленный лимит.
    Выход: 
        1 - если авто старше
        0 - если нет.
    """
    if (row['parsing_date'].year-row['productionDate']) >= year_lim:
        return 1
    return 0

In [ ]:
def iqr_analysis(series, mode=False):
    """
    Функция выводит инфорамцию о границах выборосов для признака.
    Если mode = True, возвращается верхняя и нижняя границы выбросов. Иначе, просто выводится информация на экран.
    """
    IQR = series.quantile(0.75) - series.quantile(0.25)
    perc25 = series.quantile(0.25)
    perc75 = series.quantile(0.75)

    f = perc25 - 1.5*IQR
    l = perc75 + 1.5*IQR

    if mode:
        return f, l

    print(
        "\n25-й перцентиль: {},".format(perc25),
        "\n75-й перцентиль: {},".format(perc75),
        "\nIQR: {}, ".format(IQR),
        "\nГраницы выбросов: [{f}, {l}].".format(
            f=perc25 - 1.5*IQR, l=perc75 + 1.5*IQR),
        "\n\nМинимальное значение признака: {}.".format(series.min()),
        "\nМаксимальное значение признака: {} .\n".format(series.max()))

    if series.min() < f:
        print("Найдены выбросы по нижней границе признака! Количество: {}, {}%".format(series.where(
            series < f).count(), round(series.where(series < f).count()/series.count()*100, 2)))
    if series.max() > l:
        print("Найдены выбросы по верхней границе признака! Количество: {}, {}%".format(series.where(
            series > l).count(), round(series.where(series > l).count()/series.count()*100, 2)))

In [ ]:
def get_stat_dif(column):
    """ 
    Поиск статистически значимых различий для колонки с помощью теста Стьюдента.
    """
    cols = data.loc[:, column].value_counts().index[:]
    combinations_all = list(combinations(cols, 2))

    tmp = data[data['train'] == 1]

    for comb in combinations_all:
        if ttest_ind(tmp.loc[data[data['train'] == 1].loc[:, column] == comb[0], 'price'],
                     tmp.loc[data[data['train'] == 1].loc[:, column] == comb[1], 'price']).pvalue <= 0.05/len(combinations_all):  # учли поправку Бонферони
            # print('Найдены статистически значимые различия для колонки и комбинаций', column, comb)
            pass
        else:
            print(
                'Не найдены статистически значимые различия для колонки и комбинации', column, comb)
            return column
            break

In [ ]:
def regularise(X_train, y_train):
    """ 
    Подбор параметров GradientBoosting.
    """
    max_depth = [5, 10, 15]
    n_estimators = [100, 200,1000]
    hyperparameters = dict(max_depth=max_depth, n_estimators=n_estimators)
    model = GradientBoostingRegressor()
    model.fit(X_train, y_train)

    clf = GridSearchCV(model, hyperparameters)

    best_model = clf.fit(X_train, y_train)

    best_max_depth = best_model.best_estimator_.get_params()['max_depth']
    best_n_estimators = best_model.best_estimator_.get_params()['n_estimators']

    return best_max_depth, best_n_estimators

In [ ]:
def compute_meta_feature(regr, X_train, X_test, y_train, cv):
    """ 
    Вычисление метапризнаков Stacking.
    """
    X_meta_train = np.zeros_like(y_train, dtype=np.float32)    
    splits = cv.split(X_train)
    for train_fold_index, predict_fold_index in splits:
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]
        
        folded_regr = clone(regr)
        folded_regr.fit(X_fold_train, y_fold_train)
        
        X_meta_train[predict_fold_index] = folded_regr.predict(X_fold_predict)
        
    meta_regr = clone(regr)
    meta_regr.fit(X_train, y_train)

    X_meta_test = meta_regr.predict(X_test)

    return X_meta_train, X_meta_test

def generate_meta_features(regr, X_train, X_test, y_train, cv):
    """ 
    Генерация метапризнаков Stacking.
    """
    features = [compute_meta_feature(regr, X_train, X_test, y_train, cv) for regr in tqdm(regr)]    
    stacked_features_train = np.vstack([features_train for features_train, features_test in features]).T
    stacked_features_test = np.vstack([features_test for features_train, features_test in features]).T
    return stacked_features_train, stacked_features_test


## Setup

In [ ]:
VERSION = 17
# подключил к ноутбуку внешний датасет
#DIR_TRAIN = '../input/parsing-all-moscow-auto-ru-09-09-2020/'
#DIR_TRAIN_NEW = '../input/autorucars/'
#DIR_TEST = '../input/sf-dst-car-price-prediction/'
VAL_SIZE = 0.20   # 20%

## 2. Импорт, обзор и очистка данных

In [ ]:
# Подключение при работе в Jupyter/Github
train = pd.read_csv('all_auto_ru_09_09_2020.csv')
test = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sample_submission.csv')

In [ ]:
#!ls '../input'

In [ ]:
## Подключение с Kaggle
#train = pd.read_csv(DIR_TRAIN+'all_auto_ru_09_09_2020.csv') # датасет для обучения модели
#test = pd.read_csv(DIR_TEST+'test.csv')
#sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')

In [ ]:
# Подключение при работе в Jupyter/Github
#train = pd.read_csv('all_auto_ru_09_09_2020.csv')
#test = pd.read_csv('test.csv')
#sample_submission = pd.read_csv('sample_submission.csv')

In [ ]:
print('Размерность тренировочного датасета: ', train.shape)
display(train.head(2))
print('Размерность тестового датасета: ', test.shape)
display(test.head(2))
print('Размерность тестового submission: ', sample_submission.shape)
display(sample_submission.head(2))

In [ ]:
display(train.info())
display(test.info())

In [ ]:
# Переименуем test колонки
test.rename(columns={"Привод": "drivetrain",
                     "Руль": "driveSide",
                     "Состояние": "condition",
                     "Владельцы": "ownersCount",
                     "ПТС": "tcp",
                     "Таможня": "customs",
                     "Владение": "ownershipTime",
                     "model_name": "model"}, inplace=True)

# Переименуем train колонки
train.rename(columns={"Привод": "drivetrain",
                      "Руль": "driveSide",
                      "Состояние": "condition",
                      "Владельцы": "ownersCount",
                      "ПТС": "tcp",
                      "Таможня": "customs",
                      "Владение": "ownershipTime"}, inplace=True)

Предварительный анализ test данных
Посмотрим на данные в test, которые нужно предсказывать, на особенности данных.

In [ ]:
#ProfileReport(test, title="Pandas Profiling Report for Test Dataset")

Выводы: во время последующего сбора данных и анализа обратить внимание на:

Есть пропуски в test данных - нужно попробовать заполнить
Представлено всего 12 брендов автомобилей
Признаки, которые выглядят бесполезными для моделирования - car_url (уникальный для каждой записи), priceCurrencly (одинаковый у всех)
Необходимо разобраться с признаками complectation_dict, description, equipment_dict, model_info, super_gen - посмотреть, можно ли извлечь доп. признаки
Condition - у всех "хорошее состояние", обратить внимание, если в train будут записи и другим состоянием, может влиять на выбросы по цене.
Customs - у всех "растоможетн", обратить внимание, если в train будут записи и другим состоянием, может влиять на выбросы по цене.

Сравним типы данных в test и train.
Найдем расхождения в типах данных для train и test, устраним их до анализа данных.

In [ ]:
# Создадим список с колонками, которые присутствуют и в train, и в test
cols_intersection = list(set(test.columns).intersection(train.columns))

In [ ]:
# Проверим, в каких колонках типы данных различаются у train и test
cols_type_dif = []  # создаем список с колонками, в которых типы данных отличаются
print("Найдены расхождения в типах данных для:")

for col in cols_intersection:
    if type(test[col][0]) != type(train[col][0]):
        print(
            f"\t- колонки {col}: для train - {type(train[col][0])}, для test - {type(test[col][0])}")
        cols_type_dif.append(col)

#### ownersCount

Количество владельцев автомобиля (1 - один, 2 - два, 3 - 3 и более).

In [ ]:
print("Уникальные значения для train", train.ownersCount.unique())
print("Уникальные значения для test", test.ownersCount.unique())

Преобразуем значения test в числовые (float).

In [ ]:
# Создаем словать с кодировкой значений в числовые
ownersCount_dict = {"3 или более": 3.,
                    "1\xa0владелец": 1., "2\xa0владельца": 2.}
test['ownersCount'].replace(
    to_replace=ownersCount_dict, inplace=True)  # заменяем значения в соответствии со словарем

In [ ]:
# Проверяем результаты преобразования
print("Уникальные значения для test", test.ownersCount.unique())
print("Уникальные значения для train", train.ownersCount.unique())

#### enginePower

Мощность л.с.

In [ ]:
print("Пример значения для train", train.enginePower.sample().values)
print("Пример значения для test", test.enginePower.sample().values)

Преобразуем значения test в числовые (float).

In [ ]:
# Преобразовываем значения столбца, избавляемся от постфика N12
test['enginePower'] = test['enginePower'].apply(
    lambda x: float(str(x).split(" ")[0]))

In [ ]:
# Посмотрим на результат преобразования
print("Пример значения для test", test.enginePower.sample(1).values)
print("Пример значения для train", train.enginePower.sample(1).values)

#### numberOfDoors

Количество дверей.

In [ ]:
print("Уникальные значения для train", train.numberOfDoors.unique())
print("Уникальные значения для test", test.numberOfDoors.unique())

In [ ]:
# Конвертируем значения train в int
train['numberOfDoors'] = train['numberOfDoors'].astype('Int64')

#### customs

Информация, растоможен ли автомобиль: 0 - не растоможен, 1 - растоможен.

In [ ]:
print("Уникальные значения для train", train.customs.unique())
print("Уникальные значения для test", test.customs.unique())

Поскольку далее для моделирования нам понадобится числовая переменная, то сразу переведем в числовую 0 - не растоможен, 1 - растоможен, как для train, так и для test.

In [ ]:
# Делаем преобразование значений в int (0 или 1)
train['customs'] = train['customs'].apply(lambda x: 1 if x == True else 0)
test['customs'] = test['customs'].apply(
    lambda x: 1 if x == "Растаможен" else 0)

#### condition

Состояние автомобиля: 0 - не требует ремонта , 1 - требует ремонта.

In [ ]:
print("Уникальные значения для train", train.condition.unique())
print("Уникальные значения для test", test.condition.unique())

Поскольку далее для моделирования нам понадобится числовая переменная, то сразу проведем преобразование как для train, так и для test.

In [ ]:
# Делаем преобразование значений в int (0 или 1)
train['condition'] = train['condition'].apply(lambda x: 0 if pd.isna(x) else 1)
test['condition'] = test['condition'].apply(
    lambda x: 0 if x == "Не требует ремонта" else 1)

#### modelDate

Год выпуска модели автомобиля.

In [ ]:
print("Уникальные значения для train", train.modelDate.unique())
print("Уникальные значения для test", test.modelDate.unique())

Годы - целые числа. Преобразуем train данные в int.

In [ ]:
train['modelDate'] = train['modelDate'].astype('Int64')

#### ownershipTime 

Вемя владения автомобилем. Формат хранения информации в train и test отличается.

In [ ]:
# Применяем функцию по очистке данных и приведению train к такому же виду, как в test выборке.
train['ownershipTime'] = train['ownershipTime'].apply(tenure)

In [ ]:
# Заполним пропуски в train выборке значениями 'no_data'.
train['ownershipTime'] = train['ownershipTime'].fillna('no_data')

#### mileage

Пробег

In [ ]:
print("Пример значения для train",
      train[train['mileage'].isnull() == False]['mileage'].sample(1).values)
print("Пример значения для test",
      test[test['mileage'].isnull() == False]['mileage'].sample(1).values)

#### engineDisplacement

Объем двигателя в литрах, нужно привести к общему формату во всех датасетах.

In [ ]:
print("Пример значения для train", train[train['engineDisplacement'].isnull(
) == False]['engineDisplacement'].sample(1).values)
print("Пример значения для test", test[test['engineDisplacement'].isnull(
) == False]['engineDisplacement'].sample(1).values)

Избавимся от LTR в наименовании объема двигателя и конвертируем во float.

In [ ]:
# Сконвертируем информацию об объёме двигателя из поля name
train['engineDisplacement'] = train['name'].apply(
    convert_engineDisplacement_to_float)

In [ ]:
# Сразу проверим пропуски
len(train[train['engineDisplacement'].isna()])

In [ ]:
# Пропуски для электро машин, понятно причина пропуска, обработаем позже
train[train['engineDisplacement'].isna()].fuelType.unique()

In [ ]:
# Для 'LTR' значения в test заменим на 0, т.к. это электрокары
test[test['engineDisplacement'] == ' LTR']['engineDisplacement'].replace(
    " LTR", "0.0 LTR", inplace=True)
# Сконвертируем информацию во float
test['engineDisplacement'] = test['engineDisplacement'].apply(
    convert_engineDisplacement_to_float)

### Совмещаем test, train для обработки и анализа

До совмещения test и train посмотрим, какие признаки можем извлечь из test/train, помимо колонок, которые в явном виде пересекаются между датафреймами.

In [ ]:
# Список колонок, которых нет в train, но есть в test
dif_list_test = list(set(test.columns).difference(train.columns))
print("Список колонок, которых нет в train, но есть в test:", dif_list_test)

# Посмотрим на данные в этих колонках
print("\nTEST DF:")
test[dif_list_test].sample(3)

In [ ]:
# Список колонок, которых нет в test, но есть в train
dif_list_train = list(set(train.columns).difference(test.columns))
print("Список колонок, которых нет в test, но есть в train:", dif_list_train)

# Посмотрим на данные в этих колонках
print("\nTRAIN DF:")
train[dif_list_train].sample(3)

Идеи для обогощений датасетов, чтобы получить общие данные для работы:

[x] Train: добавить колонку vendor, можно создать словарик соответсвия brand-vendor
[x] All: поскольку все данные парсились в разное время, то добавим колонку parsing_date для всех датафреймов.
В признаках super_gen и equipment_dict лежит информация о характеристиках и комплектации автомобиля, частично дублирующая информацию остальных признаков.

#### Добавляем vendor в train

In [ ]:
# Создаем словарь из вендоров по брендам
vendor_dict = test.groupby(['vendor'])['brand'].apply(
    lambda grp: list(grp.value_counts().index)).to_dict()
vendor_dict

In [ ]:
# Список брендов, которых нет в test, но есть в train
dif_list_brands = list(
    set(train.brand.unique()).difference(test.brand.unique()))
print("Список колонок, которых нет в test, но есть в train:", dif_list_brands)

In [ ]:
# Добавим в словарь vendor_dict недостающие бренды по существующим ключам
eur_append = ['PORSCHE', 'LAND_ROVER', 'JAGUAR',
              'MINI', 'RENAULT', 'OPEL', 'PEUGEOT', 'CITROEN', 'FERRARI', 'FIAT', 'SMART', 'AURUS']
jap_append = ['SUBARU', 'MAZDA', 'SUZUKI']

for brand in eur_append:
    vendor_dict['EUROPEAN'].append(brand)
for brand in jap_append:
    vendor_dict['JAPANESE'].append(brand)

In [ ]:
# Добавляем новые ключи
vendor_dict.update({'AMERICAN': ['CHEVROLET', 'CHRYSLER', 'CADILLAC', 'JEEP', 'FORD', 'DODGE', 'FISKER', 'TESLA'],
                    'ASIAN': ['HYUNDAI', 'DAEWOO', 'KIA', 'CHERY', 'SSANG_YONG', 'GEELY', 'GREAT_WALL', 'ACURA',
                             'BYD', 'SUZUKI', 'XPENG']})

In [ ]:
# Смотрим итоговый словарь
print(vendor_dict)

In [ ]:
# Создадим новый словарь для мапинга
new_map = {str(x): str(k) for k, v in vendor_dict.items() for x in v}

In [ ]:
# Создаем признак vendor в train
train['vendor'] = train.brand.copy()
train['vendor'] = train['vendor'].map(new_map)

#### Добавляем parsing_date во все датасеты

In [ ]:
# Добавляем фиксированную дату в train, берем признак из имени файла (09.09.2020)
train['parsing_date'] = '2020-09-09'
train['parsing_date'] = pd.to_datetime(train['parsing_date'])

In [ ]:
# Добавляем колонку из unixtime в test, train
test['parsing_date'] = pd.to_datetime(test['parsing_unixtime'], unit='s')
test['parsing_date'] = pd.to_datetime(test['parsing_date']).dt.floor('d')

In [ ]:
# Поскольку это служебный признак, то мы его сразу добавляем в список на удаление
# Создадим список и добавим колонки, которые планируем удалить по результатам предварительного анализа
cols_removal = ['parsing_date']

In [ ]:
# Для анализа склеиваем все датафреймы по общим колонкам, добавляем признак train
train['train'] = 1  # помечаем где у нас трейн
test['train'] = 0  # помечаем где у нас тест
# в тесте у нас нет значения цены, мы его должны предсказать, поэтому пока просто заполняем нулями
test['price'] = 0
train['sell_id'] = 0  # поле понадобавиться для сабмита

data = pd.concat([train, test], axis=0,
                 join="inner", ignore_index=True)

In [ ]:
# Посмотрим, что получилось
data.info()

In [ ]:
# Преобразуем modelDate в int
data['modelDate'] = data['modelDate'].astype('Int64')

In [ ]:
data.sample(3)

#### Проведем предварительный анализ данных.

In [ ]:
# Запустим библиотеку для предварительного анализа данных
#ProfileReport(data, title="Pandas Profiling Report for Merged Test and Train Datasets")

Выводы:

Суммарно 124064 строки, 1.2% пропусков, 0.4% дубликатов. Данные достаточно "чистые", пригодны для работы.
В датасете 27 признаков. Типы переменных: числовых - 7, категориальных - 18, неопределенный - 1.
Идеи по очистке данных в колонках и вопросы для анализа данных:

bodyType - убрать верхний и нижний регистр, сократить кол-во категорий, убрать информацию о количестве дверей, тк есть отдельный признак numberOfDoors.
color - коды перевести в цвета
fuelType - разобраться, что такое универсал
vehicleTransmission - рус и англ варианты унифицировать
numberOfDoors - разобраться с транспортом 0 дверей
driveSide - рус и англ варианты унифицировать
tcp - рус и англ варианты унифицировать
description - кандидат на удаление, но можно извлечь новые признаки
ownershipTime - кандидат на удаление, т.к. много пропусков
price - есть пропуски, т.к. целевая переменная не заполняем, а удаляем строки.
Признаки-кандидаты на удаление:

vehicleConfiguration - дублирует информацию из колонок vehicleTransmission, engineDisplacement, bodyType Удалить признак после обработки пропусков.
name - дублирует информацию из колонок engineDisplacement, vehicleTransmission, enginePower, drivetrain. Удалить признак после обработки пропусков
price - есть пропущенные значения, удалить строки из обучающей выборки.
Корреляционный анализ:
есть колинеарные признаки:
(productionDate - modelDate, mileage - modelDate/productionDate, ownersCount - modelDate/productionDate). Логично, что дата модели и дата производства связаны, также чем старше машина, тем больше пробег и тем большее количетсво владельцев авто)
`conditions, customs - некорретные коэффициенты, нужно анализировать.
Наибольшее количество пропусков в колонках ownershipCount, ownersCount. Можно попробовать предсказать количество, исходя из даты выпуска (т.к. признаки скоррелированы).

Создадим список, в который будем добавлять колонки-кандидаты на удаление из датасета.

In [ ]:
# Создадим список и добавим колонки, которые планируем удалить по результатам предварительного анализа
cols_removal.append('vehicleConfiguration')
cols_removal.append('name')

### Проведем чистку и дополнительную подготовку данных.

#### bodyType

Тип кузова.

In [ ]:
data.bodyType.value_counts()

In [ ]:
# Приведем значения к нижнему регистру, уберем ряд дубликатов в значениях
data['bodyType'] = data['bodyType'].apply(lambda x: str(x).lower())

In [ ]:
# Выполним стандартизацию типов кузовов и приведем в соответствие с настройками поиска auto.ru
data['bodyType'].replace(regex={r'внедорожник.*': 'внедорожник',
                                r'седан.*': 'седан',
                                r'хэтчбек.*': 'хэтчбек',
                                r'купе.*': 'купе',  r'тарга.*': 'тарга', r'микровэн.*': 'микровэн',
                                r'пикап.*': 'пикап', r'родстер.*': 'родстер',
                                r'универсал.*': 'универсал', r'лифтбек.*': 'лифтбек',
                                r'минивэн.*': 'минивэн', r'компактвэн.*': 'компактвэн',
                                r'лимузин.*': 'лимузин', r'фургон.*': 'фургон', r'кабриолет.*': 'кабриолет'
                                },
                         inplace=True)

In [ ]:
print("Уникальные значения типа кузова после очистки:", data.bodyType.unique())

#### vehicleTransmission

Информация про коробку передач.

In [ ]:
data.vehicleTransmission.value_counts()

In [ ]:
# Создаем колонку transmission со значениями "автомат" и "механическая"
data['transmission'] = data.vehicleTransmission.copy()

data['transmission'].replace(['AUTOMATIC', 'ROBOT', 'VARIATOR',
                              'роботизированная', 'автоматическая', 'вариатор'], "AT", inplace=True)
data['transmission'].replace(
    ['MECHANICAL', 'механическая'], "MT", inplace=True)

In [ ]:
# В колонке vehicleTransmission почистим данные значениями 'робот', 'автоматическая', 'вариатор' или "механическая"
data['vehicleTransmission'].replace(
    ['AUTOMATIC', 'автоматическая'], "автоматическая", inplace=True)
data['vehicleTransmission'].replace(
    ['ROBOT', 'роботизированная', ], "робот", inplace=True)
data['vehicleTransmission'].replace(
    ['VARIATOR', 'вариатор'], "вариатор", inplace=True)
data['vehicleTransmission'].replace(
    ['MECHANICAL', 'механическая'], "механическая", inplace=True)

In [ ]:
# Проверим, какие значения получились
print(data['transmission'].unique())
print(data['vehicleTransmission'].unique())

#### color

Цвет автомобиля.

In [ ]:
# Посмотрим на представленные цвета
data.color.unique()

In [ ]:
# Создаем словать с кодировкой значений в цвета
color_dict = {'040001': 'чёрный', 'EE1D19': 'красный', '0000CC': 'синий', 'CACECB': 'серебристый', '007F00': 'зелёный',
              'FAFBFB': 'белый', '97948F': 'серый', '22A0F8': 'голубой', '660099': 'пурпурный', '200204': 'коричневый',
              'C49648': 'бежевый', 'DEA522': 'золотистый', '4A2197': 'фиолетовый', 'FFD600': 'жёлтый',
              'FF8649': 'оранжевый', 'FFC0CB': 'розовый'}

data['color'].replace(to_replace=color_dict, inplace=True)

In [ ]:
# Проверяем результат
data.color.unique()

#### driveSide

Расположение руля

In [ ]:
data.driveSide.unique()

In [ ]:
# Заменим анлоязычные значения на русские
data['driveSide'] = data['driveSide'].map(
    {'RIGHT': 'Правый', 'LEFT': 'Левый', 'Правый': 'Правый', 'Левый': 'Левый'})

In [ ]:
# Уберем верхний регистр
data['driveSide'] = data['driveSide'].str.lower()

In [ ]:
# Проверяем результат
data.driveSide.unique()

#### tcp

Оригинал ПТС

In [ ]:
data.tcp.unique()

In [ ]:
# Заменим анлоязычные значения на русские
data['tcp'] = data['tcp'].map(
    {'ORIGINAL': 'Оригинал', 'DUPLICATE': 'Дубликат', 'Оригинал': 'Оригинал', 'Дубликат': 'Дубликат'})

In [ ]:
# Уберем верхний регистр
data['tcp'] = data['tcp'].str.lower()

In [ ]:
# Проверяем результат
data.tcp.unique()

### Определим количество дубликатов в данных в части train.

In [ ]:
print("Количество дубликатов строк в train части датафрейма:",
      data[data.train == 1].duplicated().sum())

In [ ]:
# Удалим дубликаты из датасета
data.drop_duplicates(inplace=True)

In [ ]:
print(f"Для анализа осталось {len(data)} записей.")

### Посмотрим вцелом на наличие пустых значений и определим стратегию работы с ними.

In [ ]:
# Запускаем функцию вывода всех пустых значений
print("Пропуски для TEST+TRAIN датафрейов.\n")
intitial_eda_checks(data, 0)

In [ ]:
print("Пропуски для TEST датафрейма.\n")
intitial_eda_checks(data[data.train == 0], 0)

В 17 столбцах присутствуют пропуски.
Test:
[x] tcp: пропуск в одной записи, посмотреть на нее детально, попробовать восстановить или заполнить модой
[x] ownershipTime: много пропусков (около 65%), проанализировать с чем скоррелирован признак, попробовать заполнить. Если заполнить не получится, то возможно, удалить из признаков для моделирования.
All:
[x] ownersCount - можно построить мапинг с медианой/модой по группам в зависимости от года выпуска авто.
[x] price - значения с пропусками удалить из данных
[x] tcp - поисследовать пропуски и подумать над способами заполнения.
[x] есть ряд столбцов, где присутсвует одинаковое количество (1) пропусков. Проверить гипотезу, что пропуски находятся в одной и той же строке.

#### price

In [ ]:
# Проверим, что price отсуствует именно в train данных и мы не затроним test
data[data.price.isnull() == True]['train'].value_counts()

In [ ]:
# Удаляем строки с price = NAN
data.dropna(subset=['price'], inplace=True)

Проверим строки, где есть пропуски по drivetrain, vehicleTransmission, enginePower, transmission, vehicleConfiguration, numberOfDoors, name, modelDate.

In [ ]:
# Посмотрим на запись, чтобы проверить гипотезу
data[data.modelDate.isnull() == True]

In [ ]:
# Гипотеза подтвердилась, исключим строку из анализа
row_index = data[data.modelDate.isnull() == True].index[0]
data.drop([row_index], inplace=True)

#### tcp

Проверим гипотезу, что:
tcp пустой у новых машин, tcp пустой там, где это дубликат.

In [ ]:
# Посмотрим на примеры таких данных
data[data.tcp.isnull() == True].sample(3)

In [ ]:
# Сгруппируем данные по году выпуска и выведем медианный пробег и кол-во владельцев
data[data.tcp.isnull() == True].groupby('productionDate')[
    'mileage', 'ownersCount'].median()

Действительно, выглядит так, что NAN - для новых машин и одной машины с теста.

In [ ]:
# Для одной записи из test присваиваем значение tcp "дубликат"
data.tcp.loc[data[(data.tcp.isnull() == True) & (
    data.train == 0)]['tcp'].index[0]] = 'дубликат'

In [ ]:
# Смотрим, какой mileage у оставшихся записей. Удостоверимся, что это авто без пробега
data[data.tcp.isnull() == True].mileage.value_counts()

In [ ]:
# Заполняем значением "оригинал" для авто без пробега
data.tcp.fillna('оригинал', inplace=True)

In [ ]:
# Проверяем, что получилось
data.tcp.value_counts()

#### description

Пропуски заполнить проблематично, перед постраением модели признак удалим, пока оставим. Идея для нового признака - наличие описания (да/нет).

In [ ]:
# добавим колонку в список на удаление перед построением модели
cols_removal.append('description')

#### ownershipTime

Попробуем восстановить информацию о времени владения, исходя из информации в productionDate.

In [ ]:
# Создаем новый столбец с количеством месяцев владения
data['ownershipTimeMonths'] = data['ownershipTime'].apply(num_of_months)

In [ ]:
# Пока заполняем пропуски no_data, далее решаем, что сделать с признаком
data['ownershipTime'] = data['ownershipTime'].fillna('no_data')

In [ ]:
# Смотрим, что получилось
data[data.ownershipTime != 'no_data'][[
    'ownershipTime', 'ownershipTimeMonths']].sample(5)

In [ ]:
# Заполним нулями признак для новых автомобилей
data['ownershipTimeMonths'].loc[(data['mileage'] == 0) & (
    data['ownershipTimeMonths'].isnull() == True)] = 0

In [ ]:
# Добавим исходную колонку в список на удаление перед построением модели, т.к. пропусков очень много
cols_removal.append('ownershipTime')
cols_removal.append('ownershipTimeMonths')

#### ownersCount

Количество владельцев.

In [ ]:
# Все пропуски по значению находятся в train части
data[data.ownersCount.isnull() == True].train.value_counts()

In [ ]:
# Посмотрим по данным с незаполненным количетсвом владельцев медианные значения метрик.
data[data.ownersCount.isnull() == True].groupby('productionDate').median()

In [ ]:
# Посмотрим на уникальные значения пробега по отфильтрованным данным.
data[data.ownersCount.isnull() == True]['mileage'].describe()

In [ ]:
# Владельцы не заполнены, потому что это новые авто. Заполняем пропуски 0.
data['ownersCount'].fillna(0, inplace=True)

In [ ]:
# Приведем значение признака из float в int
data['ownersCount'] = data['ownersCount'].astype('int64')

### Детальный анализ признаков
Группировка признаков на категориальные, бинарные и числовые
Посмотрим, какие признаки могут относиться к категориальным, бинарным, числовым.

In [ ]:
# Посмотрим, сколько уникальных значений в признаках
print("Количество уникальных категорий в признаках.")
data.nunique(dropna=False)

In [ ]:
# Рассмотрим пример признака
print("Выведем пример значений признаков одной из записей датасета.")
data.loc[11]

In [ ]:
# Посмотрим, на колонки, которые планируем впоследствии удалить, чтобы не включать их в анализ
print("Признаки для последующего удаления:", cols_removal)

In [ ]:
# Создадим списки с разными категориями признаков
# бинарные признаки
bin_cols = ['condition', 'customs', 'driveSide', 'transmission', 'tcp']

# категориальные переменные
cat_cols = ['bodyType', 'brand', 'color', 'fuelType', 'drivetrain',
            'model', 'vendor', 'vehicleTransmission', 'numberOfDoors', 'ownersCount']

# числовые переменные
num_cols = ['modelDate', 'productionDate',
            'enginePower', 'mileage', 'engineDisplacement']

# сервисные переменные
service_cols = ['train', 'sell_id', 'parsing_date']

# целевая переменная
target_col = ['price']

all_cols = bin_cols + cat_cols + num_cols + service_cols + target_col

print("Кол-во столбцов, для дальнейшей работы после предварительного анализа:", len(all_cols))

### Распределние численных признаков.

In [ ]:
# Построим распределение основных числовых признаков

print("Диаграмы распределения числовых признаков, взаимосвязь с целевой переменной")

fig, axes = plt.subplots(5, 3, figsize=(30, 40))
plt.subplots_adjust(wspace=0.5)
axes = axes.flatten()
i = 0

for col in num_cols:
    sns.distplot(data[col], ax=axes[i])
    i = i + 1
    sns.boxplot(data[col], ax=axes[i])
    i = i + 1
    sns.scatterplot(data=data[data['train'] == 1],
                    x=col, y="price", ax=axes[i])
    i = i + 1

In [ ]:
print("Основные статистики для числовых признаков.")
display(data[num_cols].describe())

Выводы:

После построения гистограмм стало очевидно, что распределения основных числовых переменных имеют тяжёлый левый или правый хвост, влияет на разбежку между средним и медианой:
для того, чтобы сделать распределение данных переменных более нормальным, можно работать с логарифмированными величинами этих переменных
поработать с выбросами
подумать про группировку признаков по категориям.
Распределение modelDate, productionDate очень схожи:
проверить признаки на мультиколлинеарность во время корреляционного анализа
видно, что в среднем productionDate чуть позже, чем modelDate во времени, т.е. производство приосходит чуть с запозданием, что логично. Можно добавить новый признак - насколько новая модель, т.е. через сколько лет после появления модели был выпущен автомобиль.
mileage: есть пик в 0 - признак нового авто.
enginePower: 11 лс - что это за авто? Поисследовать
engineDisplacement: основная масса авто - 2.0 л, есть разброс.
Взаимосвязь признаков с таргетом:
наблюдается прямая корреляция цены от года модели и года выпуска с исключениями для особо раритетных автомобилей.
наблюдается обратная корреляция цены от пробега авто
цена от мощности вдигателя и объема двигателя зависит, но не так явно, как от других переменных.

#### Корреляционный анализ.
Оценим корреляцию Пирсона для непрерывных переменных. Cильная корреляция между переменными вредна для линейных моделей из-за неустойчивости полученных оценок.

In [ ]:
# Построим матрицу корреляций
heatmap = sns.heatmap(data[num_cols + target_col].corr(), vmin=-1,
                      vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('Матрица корреляций числовых и целевой переменных')
plt.show()

Вывод:

Взаимосвязь пар числовых признаков по Пирсону достаточно сильная для productionDate, modelDate. При этом, у productionDate чуть большая корреляция с целевой переменной. Удалим признак modelDate.
Достаточно сильная корреляция между productionDate и mileage - чем меньше год выпуска, тем больше пробег
engineDisplacement и enginePower достаточно сильно скоррелированы (0.84). Для моделирования оставим один признак - enginePower, т.к. он сильнее влияет на price.
У всех числовых признаков достаточно высокая корреляция с целевой переменной, это хорошо.

In [ ]:
# Добавляем modelDate в список колонок на удаление.
cols_removal.append('modelDate')
cols_removal.append('engineDisplacement')

#### Дополнительный анализ числовых признаков
Посмотрим более пристально на признаки, которые не исключили из анализа.

#### mileage

Дополнительно исследуем признак.

In [ ]:
# Посмотрим на топ 5 годов выпуска авто
fig, ax = plt.subplots(figsize=(10, 5))

for x in (data['productionDate'].value_counts())[0:7].index:
    data['mileage'][data['productionDate'] == x].hist(bins=50)

ax.set_xlim(0, 700000)
ax.set_ylim(0, 1650)

plt.title("Распределение mileage по productionDate")
plt.show()

Получается, что mileage имеет достаточно нормальное распределение, не с таким уж сильным правым смещением, если смотреть на графики по годам производства, а не на все числа. Чем старше автомобиль, тем больше пробег и мы наблюдаем смещение.

Чтобы скорректировать признак и сделать его более информативным, создадим новый признак в разделе создания признаков: km_per_year - показывает, сколько км в год проезжал автомобиль. Вычисляется как mileage/productionDate(years) или mileage/ownershipTime. Решить, какой метод использовать.

#### enginePower

Нужно поисследовать странное минимальное значение мощности.

In [ ]:
print("Значение мощности двигателя (л.с.) на TEST")
display(data[data.train == 0]['enginePower'].describe())

print("\nЗначение мощности двигателя (л.с.) на TRAIN")
display(data[data.train == 1]['enginePower'].describe())

Видим, что в test значение минимума больше, а максимума меньше. Посмотрим, что это за авто и сколько их.

In [ ]:
print("Строки, в которых мощность двигателя меньше 30 л.с.")
data[data.enginePower < data[data.train == 0]['enginePower'].min()]

In [ ]:
print("Строки, в которых мощность двигателя более 639 л.с.")
data[data.enginePower > data[data.train == 0]['enginePower'].max()]

Таких строк немного, удалять строки нельзя, т.к. необходимо строить предсказания по большим значениям enginePower.

#### productionDate/modelDate

Посмотрим повнимательнее на признак и попробуем проанализировать поглубже.

In [ ]:
print("Значение года производства авто на TEST")
display(data[data.train == 0]['productionDate'].describe())

print("\nЗначение года производства авто на TRAIN")
display(data[data.train == 1]['productionDate'].describe())

Минимальные значения, максимальные, медианные и среднии достаточно близки.

Посмотрим на раритетные автомобили (30 летние машины, согласно классификации FIVA (Fédération Internationale des Véhicules Anciens).

In [ ]:
# Посмотрим, сколько таких записей
print(
    f"В датасете содержится {len(data[data.productionDate < (data['parsing_date'].max().year-30)]['productionDate'])} раритетных автомобилей.")

In [ ]:
# Детальнее посмотрим, как падает цена в зависимости от productionDate

fig, ax = plt.subplots(figsize=(10, 5))

ax.set_xlim(1970, 2021)
ax.set_ylim(0, 30000000)
ax.set_xticks(np.arange(1970, 2022, 3))

sns.scatterplot(data=data[data['train'] == 1], x='productionDate', y="price")

plt.title("Распределение целевой переменной в зависимости от productionDate\n")
plt.show()

Можно найти много информации, что цена автомобиля значительно падает после 3-5 лет эксплуатации. На графиках даная динамика тоже прослеживается. Идеи для новых признаков:

prodDate_3Y - признак, что автомобилю уже 3 года
prodDate_5Y - признак, что автомобилю уже 5 лет.

### Посмотрим на распределение признаков.

In [ ]:
# Построим распределение основных бинарных и категориальных признаков
print("Распределение бинарных и категориальных признаков. Нажимите дважды для увелечения.")

fig, axes = plt.subplots(4, 4, figsize=(35, 35))
plt.subplots_adjust(wspace=0.5)
axes = axes.flatten()
i = 0

for col in (bin_cols + cat_cols):
    sns.histplot(data=data, x=data[col], ax=axes[i],
                 stat='count', bins=data[col].nunique())
    plt.tight_layout()
    plt.xticks(rotation=45)
    plt.title(col)
    i = i + 1

In [ ]:
# Построим график boxplot
print("Boxplot нечисловых признаков и их зависимость от цены.")
for col in (bin_cols + cat_cols):    
    get_boxplot(col)

Выводы по всем графикам:

Полностью сбалансированные признаки отсутсвуют.
Особо несбалансированные признаки:

[x] driveSide - левосторонние машины в большинстве: выяснить, есть ли правосторонние машины в тесте. Если нет, то удалить признак
[x] bodyType - некоторые категории представлены небольшим количеством машин, но boxplot показывает значительные различия и распределении по ценам. Идея схлопнуть малочисленные категории в other кажется не очень хорошей, т.к. разброс медианных цен очень большой для этих категорий. Оставлим пока признак, как есть.
[x] brand - много категорий, заметно, что есть массовые, среднепопулярные и редкие бренды авто. Поисследовать дополнительно и подумать над созданием новых признаков (престижные авто/люкс, популярные и т.д.)
[x] color - есть популярные цвета (черный, белый, серый, серебристый, синий) и редкие. Посмотреть дополнительно и создать новый признак о популярности цвета
[x] fuelType - есть типы топлива, которые в явном меньшинстве. Поисследовть и подумать, стоит ли делать группировку непопулярных типов топлива
[x] tcp - несбалансированный признак, но пока оставляем в модели
[x] model - очень много уникальных категорий, подумать, можно ли как-то доработать признак. Подумать про объединение brand + model
[x] numberOfDoors - малое количество машин в 0-3 - изучить детальнее

Сбалансированные признаки с заметно превалирующим классом:

transmission - автомат превалирует
drivertrain - полный привод встречается чаще всего
vendor - большее количество автомобилей европейского региона
ownersCount - привалирует 3 и более.

Неинформативные признаки: conditions, customs - после манипуляций с данными в признаках осталось только одно значение. Удаляем из анализа.

Зависимость с целевой переменной:

driveSide: авто с правосторонним рулем в среднем дешевле машин с левосторонним рулем
transmission: авто с АТ коробкой намного дороже MT, как и сам диапазон цен
bodyType: признак, который значительно влияет на распределение цен
brand: большая разбежка цен от бренда. Выделяются престижные авто (porche, Cadillac, bmw, and Rover, Lexus и др), а есть дешевый сегмент (азиатские авто - Cherry, Daewoo, Great wall и др.). Также видны бренды, которые выпускают дорогие авто, но и есть модели для более дешевого сегмента.
color: цены зависят от цвета, но большие цены представлены у цветов, количество авто по которым больше.
fuelType: очень дорогие машины электро и дизель, возможно выделить отдельный признак, что машина “электрокар”
drivetrain: полноприводные машины дороже всех, заднеприводные машины в среднем дешевле переднеприводных
tcp: авто с дубликатом ПТС дешевле чем те, что с оригиналом
model - данных много, но видно, что присутствую колебания цены в зависимости от модели
vendor: в среднем, европейские и японские машины дороже американских и азиатских
vehicleTransmission: в среднем разновидности автоматов особо не влияют на цену, проверить значимость признака тестом Стьюдента. Потенциально на исключение.
numberOfDoors: в среднем самые дорогие авто - 2-х дверные, затем 5-дверные.
ownersCount: чем больше владельцев, тем ниже средняя цена авто.

In [ ]:
# Добавляем modelDate в список колонок на удаление.
cols_removal.append('condition')
cols_removal.append('customs')

Дополнительный анализ бинарных и категориальных признаков
Посмотрим более пристально на признаки, которые не исключили из анализа и по которым возникли вопросы/требуется дополнительное исследование после предварительного анализа.

#### driveSide

Левосторонние машины в большинстве, выясним, есть ли правосторонние машины в test. Если нет, то удалим признак.

In [ ]:
# Проверяем распределение признака в test
test.driveSide.value_counts(normalize=True)

4% данных в тесте правосторонние. Оставляем как есть.

#### brand

Марка машины.

In [ ]:
print(
    f'Список категорий brand в test, которые не представлены в train: \n{list(set(data[data.train == 0].brand.unique()).difference(data[data.train == 1].brand.unique()))}')
print(
    f'\nСписок зачений brand в train, по которым не нужно делать предсказания в test: \n{list(set(data[data.train == 1].brand.unique()).difference(data[data.train == 0].brand.unique()))}')

In [ ]:
# Посмотрим внимательно на графики еще раз
analyze_cat_cols('brand')

Можно создать новые признаки:

Флаг популярного авто: за ориентир взять значение data['brand'].value_counts() > data['brand'].value_counts().values.mean().
Обычно на цену сильно влияет класс автомобиля (brand + bodyType), попробовать добавить признак с помощью метода главных компонент.
Также в train присутсвует значительно большее количество брендов, чем в test. Учитывать при обработке выбросов.

#### model

Модель машины.

In [ ]:
print(
    f'Список категорий brand в test, которые не представлены в train: \n{list(set(data[data.train == 0].model.unique()).difference(data[data.train == 1].model.unique()))}')
print(
    f'\nСписок зачений brand в train, по которым не нужно делать предсказания в test: \n{list(set(data[data.train == 1].model.unique()).difference(data[data.train == 0].model.unique()))}')

Вывод:

признак пока оставляем неизменным, можно добавить новый признак популярности модели среди брендов.
В test есть модели, по которым нет данных в train - возможны ошибки в предсказании.

#### color

Есть популярные цвета и редкие. Создадим новый признак популярности цвета.

In [ ]:
print(
    f'Список категорий brand в test, которые не представлены в train: \n{list(set(data[data.train == 0].color.unique()).difference(data[data.train == 1].color.unique()))}')
print(
    f'\nСписок зачений brand в train, по которым не нужно делать предсказания в test: \n{list(set(data[data.train == 1].color.unique()).difference(data[data.train == 0].color.unique()))}')

In [ ]:
# Посмотрим внимательно на графики еще раз
analyze_cat_cols('color')

Создадим признак популярного авто: за ориентир взять значение data['color'].value_counts() > data['color'].value_counts().values.mean().

#### fuelType

Есть типы топлива, которые в явном меньшинстве. Поисследовть и подумать, стоит ли делать группировку непопулярных типов топлива.

In [ ]:
print(
    f'Список зачений по fuelType в трейне:= {list(data[data.train == 0].fuelType.unique())}')
print(
    f'Список зачений по fuelType в тесте:= {list(data[data.train == 1].fuelType.unique())}')

In [ ]:
# Посмотрим внимательно на графики еще раз
analyze_cat_cols('fuelType')

In [ ]:
# Посмотрим, какие бренды представлены в "электро" типе топлива в трейне и тесте
data[data['fuelType'] == 'электро'].groupby(
    ['train', 'brand'])['bodyType'].count()

Выводы:

Перечень категорий цветов идентичен в train и test.
Категории для данного признака оставим, как есть, т.к. сильно влияет на распределение цен.
[ ] идея для генерации нового признака - добавить флаг "электрокар".
Использовать информацию по распределению типа топлива-бренд между трейн и тестом для обработки выбросов.

#### numberOfDoors

Во время предварительного анализа появилась идея изучить значение 0 - выброс/ошибка или валидная запись.

In [ ]:
# Посмотрим на записи с кол-вом дверей менее 2
data[data.numberOfDoors < 2]

Все в порядке - это раритетные машины, более того 1 из test, 1 из train, оставляем.

#### description

Посмотрим на признак повнимательнее и подумаем, что мы можем из него извлечь.

In [ ]:
# Описание в train
data[data.train == 1]['description'].iloc[25]

In [ ]:
# Описание в test
data[data.train == 0]['description'].iloc[18]

Для анализа всего текста времени нет, но можно попробовать извлечь признаки количества слов в объявлении.

#### Преобразование бинарных переменных в числа
Чтобы алгоритмы машинного обучения могли работать с категориальными данными, их нужно преобразовать в числа. Применим LabelEncoder ко всему набору бинарных переменных.

In [ ]:
# Для бинарных признаков мы будем использовать LabelEncoder
label_encoder = LabelEncoder()

for column in bin_cols:
    data[column] = label_encoder.fit_transform(data[column])

# Убедимся в преобразовании
data[bin_cols].sample(6)

#### Анализ целевой переменной price

In [ ]:
# Посмотрим на основные статистики
print(data[data.train == 1]['price'].describe())

In [ ]:
# Посмотрим на распределение целевой переменной
fig, ax = plt.subplots(figsize=(10, 5))
sns.histplot(data=data[data.train == 1], x='price')
plt.title("Распределение целевой переменной \n")
plt.show()

Распределение цены с очень сильным правым хвостом, попробуем добавить доп. целевой признак, взяв логорифм от цены.

#### price_log
Логарифм от цены.

In [ ]:
# Добавляем новый признка
data['price_log'] = data['price'].apply(lambda x: np.log(x))

# Добавим новый признак в список целевых
target_col.append('price_log')

In [ ]:
# Посмотрим, как изменилось распределение целевой переменной
fig, ax = plt.subplots(figsize=(10, 5))
sns.histplot(data=data[data.train == 1], x='price_log')
plt.title("Распределение целевой переменной \n")
plt.show()

Вывод: распределение выглядит, как нормальное. Используем для моделирования.

### Идеи по генерации новых признаков по результатам EDA:

[x] km_per_year - показывает, сколько км в год проезжал автомобиль. Вычисляется, как mileage/productionDate(years) или mileage/ownershipTime. Решить, какой метод использовать
[x] carNovelty - показывает, через сколько лет после выхода модели был выпущен автомобиль, т.е. modelDate - productionDate. Гипотеза, что чем меньше это число, тем выше будет цена
[x] `rarity - признак, показывающий, что автомобиль раритетный (ему более 30 лет)
[x] prodDate_3Y - признак, что автомобилю уже 3 года
[x] prodDate_5Y - признак, что автомобилю уже 5 лет
[x] colorPopular - признак для обозначения, что авто популярного цвета: 1 - популярного, 0 - непопулярного
[x] brandPopular - признак для обозначения, что авто популярной марки: 1 - популярного, 0 - непопулярного
[x] modelPopular - признак для обозначения, что авто популярной модели в рамках бренда: 1 - популярного, 0 - непопулярного

In [ ]:
# Для упрощения повторного анализа для новых признаков добавим списки
cat_cols_new = []
num_cols_new = []
bin_cols_new = []

#### km_per_year

Пробег авто км/год.

In [ ]:
# Добавим дополнительный признак 'km_per_year', который равен пробегу, поделенному на возраст авто,
data['km_per_year'] = data.apply(km_per_year, axis=1)

In [ ]:
print("Диаграмы распределения km_per_year признака, взаимосвязь с целевой переменной")

fig, axes = plt.subplots(1, 3, figsize=(20, 5))
plt.subplots_adjust(wspace=0.5)
axes = axes.flatten()

sns.distplot(data['km_per_year'], ax=axes[0])
sns.boxplot(data['km_per_year'], ax=axes[1])
sns.scatterplot(data=data[data['train'] == 1],
                x='km_per_year', y="price", ax=axes[2])
plt.show()

Вывод: все равно есть правых хвост. Нужно или убрать выбросы, или попробовать логарифмирование.

In [ ]:
# добавляем новый признак к числовым
num_cols.append('km_per_year')

In [ ]:
num_cols_new.append('km_per_year')

#### carNovelty

Новизна машины: через сколько лет после появления модели авто была произведена машина.

In [ ]:
# Добавим'dateModelProdDiff', равный разнице между годом выпуска авто и годом начала производства модели
data['carNovelty'] = data['productionDate'] - data['modelDate']

In [ ]:
# Посмотрим, что получилось
data[['productionDate', 'modelDate', 'mileage',
      'km_per_year', 'carNovelty']].sample(5)

In [ ]:
# Заменим отрицательные значение на 0
data['carNovelty'] = data['carNovelty'].apply(lambda x: 0 if x < 0 else x)

In [ ]:
# добавляем новый признак к числовым
num_cols.append('carNovelty')
num_cols_new.append('carNovelty')

#### prodDate_3Y, prodDate_5Y

prodDate_3Y - признак, что автомобилю уже 3 года
prodDate_5Y - признак, что автомобилю уже 5 лет.

In [ ]:
# Добавляем новые признаки
data['prodDate_3Y'] = data.apply(prod_date_range, year_lim=3, axis=1)
data['prodDate_5Y'] = data.apply(prod_date_range, year_lim=5, axis=1)

In [ ]:
# Проверяем результат
data[['prodDate_3Y', 'prodDate_5Y',
      'productionDate', 'parsing_date']].sample(5)

In [ ]:
# Добавляем новые признаки к бинарным
bin_cols.append('prodDate_3Y')
bin_cols.append('prodDate_5Y')
bin_cols_new.append('prodDate_3Y')
bin_cols_new.append('prodDate_5Y')

#### brandPopular

[x] флаг популярного авто: за ориентир взять значение data['brand'].value_counts() > data['brand'].value_counts().values.mean() или добавить соотношение кол-ва машин бренда к общему кол-ву объявлений
[ ] обычно на цену сильно влияет класс автомобиля (brand + bodyType) попробовать добавить признак с помощью метода главных компонент

In [ ]:
# Создадим датафрейс со значениями количества автомобилей в выборке по маркам
brand_df = pd.DataFrame(data['brand'].value_counts())
# Среднее по всем значениям бренда
mean_brand = data['brand'].value_counts().values.mean()
# Добавляем признак во временный датафрейм
brand_df['brandPopular'] = brand_df.brand.apply(
    lambda x: 1 if x >= mean_brand else 0)
# Удалим ненужный столбец
brand_df.drop('brand', axis=1, inplace=True)
# Создаем словарь популярности брендов
dict_brand = brand_df.to_dict()['brandPopular']
# Создаем новый признак
data['brandPopular'] = data.brand.copy()
data['brandPopular'].replace(to_replace=dict_brand, inplace=True)
# Проверяем результат
data[['brand', 'brandPopular']].sample(5)

In [ ]:
# Добавляем новый признак к бинарным
bin_cols.append('brandPopular')
bin_cols_new.append('brandPopular')

#### modelPopular

Добавить признак популярности модели.

In [ ]:
# Создаем пустой словарь всех моделей
model_dict = {}

for item in data.brand.unique():
    model_df = pd.DataFrame(data[data.brand == item]['model'].value_counts())
    # Среднее по всем значениям моделей бренда
    mean_model = data[data.brand == item]['model'].value_counts().values.mean()
    # Добавляем признак во временный датафрейм
    model_df['modelPopular'] = model_df.model.apply(
        lambda x: 1 if x >= mean_model else 0)
    # Удалим ненужный столбец
    model_df.drop('model', axis=1, inplace=True)
    # Создаем словарь популярности брендов
    dict_model_per_brand = model_df.to_dict()['modelPopular']
    model_dict.update(dict_model_per_brand)

# Создаем новый признак
data['modelPopular'] = data.model.copy()
data['modelPopular'].replace(to_replace=model_dict, inplace=True)
# Проверяем результат
data[['model', 'modelPopular']].sample(5)

In [ ]:
# Добавляем новый признак к бинарным
bin_cols.append('modelPopular')
bin_cols_new.append('modelPopular')

#### colorPopular

Создадим признак популярного цвета авто. Популярным считаем цвет, который встречается чаще, чем среднее по всем значениям распределения цвета в выборке:

1 - популярный цвет
0 - непопулярный цвет.

In [ ]:
# Создадим отдельный датафрейм color_df со столбцами color, counts, colorPopularity
color_df = pd.DataFrame(data['color'].value_counts()).reset_index()

# Среднее по всем значениям цвета
mean = color_df.color.mean()

# Добавляем признак
color_df['colorPopular'] = color_df.color.apply(lambda x: round(x/mean, 2))

# Удаляем лишние колонки и переименовываем
color_df.drop('color', axis=1, inplace=True)
color_df.rename(columns={"index": "color"}, inplace=True)

# Посмотрим на результат
print("Относительная частота цвета по всей выборке.")
display(color_df)

# Мержим с датасетом
data = pd.merge(data, color_df, on="color", how="left")

# Выведем на экран итоговые пример
print("Пример данных в датафрейме.")
data[['color', 'colorPopular']].sample(5)

In [ ]:
# Вариант не числового а бинарного кодирования популярности авто
data['colorPopular'] = data['colorPopular'].apply(lambda x: 1 if x >= 1 else 0)

Вариант бинарного кодирования оказал лучшее влияние на метрику качества, чем числовые значения долей цвета автомобиля. Оставляем признак в бинарном виде.

In [ ]:
data[['color', 'colorPopular']].sample(5)

In [ ]:
# Добавляем новый признак к бинарным
bin_cols.append('colorPopular')
bin_cols_new.append('colorPopular')

#### description_words_count

Признак количества слов в описании.

In [ ]:
# Заполним пропуски
data['description'] = data['description'].fillna('[]')
# Создаем новый признак количества слов в описании
data['description_words_count'] = data['description'].apply(
    lambda x: len(x.split()))

In [ ]:
# Добавляем новый признак к числовым
num_cols.append('description_words_count')
num_cols_new.append('description_words_count')

#### Посмотрим на распределение новых признаков

In [ ]:
print("Диаграмы распределения новых числовых признаков, взаимосвязь с целевой переменной")

fig, axes = plt.subplots(len(num_cols_new), 3, figsize=(30, 20))
plt.subplots_adjust(wspace=0.5)
axes = axes.flatten()
i = 0

for col in num_cols_new:
    sns.distplot(data[col], ax=axes[i])
    i = i + 1
    sns.boxplot(data[col], ax=axes[i])
    i = i + 1
    sns.scatterplot(data=data[data['train'] == 1],
                    x=col, y="price", ax=axes[i])
    i = i + 1

In [ ]:
# Посмотрим внимательно на графики еще раз
for col in (bin_cols_new+cat_cols_new):
    print("Графики для переменной ", col)
    analyze_cat_cols(col)
    print("\n")

Выводы:

У числовых признаков опять тяжелый правый хвост. Попробовать работу с выбросами и логарифмирование признаков.
Визуально добавленные признаки статистически значимы и оказывают влияние на формирование цены. Проверим это на этапе отбора признаков.

### Обработка выбросов
Признаки проанализированы, новые фичи созданы. Проведем поиск, анализ и обработку выбросов для числовых и категориальных признаков. Бинарные признаки не смотрим, т.к. мы их анализировали ранее и в них содержатся допустимые значения [0, 1].

In [ ]:
# Проанализируем выбросы для категориальных и номинативных признаков
print("Отчет о наличии выбросов в датасете.\n")
for col in (num_cols + target_col):
    print("\nПризнак: ", col)
    iqr_analysis(data[col])
    print("-" * 100)

Оставляем выбросы по признакам modelDate, productionDate, carNovelty. По остальным выбросам проведем анализ и обработку.

Посмотрим детальнее на признаки, где были найдены выбросы. Основная идея: сравнить мин/макс границы по test датасету и удалить лишние данные, если их объем не очень большой.

#### enginePower

In [ ]:
# Посмотрим на максимальные значения признака в test
test.enginePower.max()

In [ ]:
# Посмотрим на распределение выбросов
fig, ax = plt.subplots(figsize=(10, 5))
sns.histplot(data=data[(data.enginePower > 351)], x='enginePower', hue = 'train')
plt.title("Распределение выбросов EnginePower в датасете \n")
plt.show()

In [ ]:
# Посмотрим на распределение выбросов
fig, ax = plt.subplots(figsize=(10, 5))
sns.regplot(data=data[(data.enginePower > 351) & (
    data.train == 1)], x="enginePower", y="price")
plt.title("Зависимость price от EnginePower в train на выбросах по EnginePower \n")
plt.show()

Выбросы по признаку представлены как в train, так и в test части. Удалять строки нельзя, т.к. необходимо строить предсказания по большим значениям enginePower.
Цена варьируется и увеличивается с увеличением мощности двигателя, заменять выбросы на другую величину не стоит.
Можно попробовать логарифмировать переменную.

#### mileage

In [ ]:
# Посмотрим на максимальные значения признака в test
test.mileage.max()

In [ ]:
# Посмотрим на распределение выбросов
fig, ax = plt.subplots(figsize=(10, 5))
sns.histplot(data=data[(data.mileage > 386527)], x='mileage', hue = 'train')
plt.title("Распределение выбросов mileage в датасете \n")
plt.show()

In [ ]:
# Посмотрим на распределение выбросов
fig, ax = plt.subplots(figsize=(10, 7))
sns.regplot(data=data[(data.mileage > 386527) & (
    data.train == 1)], x="mileage", y="price")
plt.title("Зависимость price от mileage в train на выбросах по mileage \n")
plt.show()

Выбросы по признаку представлены как в train, так и в test части. Удалять строки нельзя, т.к. необходимо строить предсказания по большим значениям mileage.
Цена варьируется и снижается с увеличением пробега, но есть и увеличение цены на макимальных значениях пробега, заменять выбросы на другую величину не стоит.
Можно попробовать логарифмировать переменную.

#### engineDisplacement

Признак позднее исключается из анализа.

In [ ]:
# Посмотрим на максимальные значения признака в test
print("Максимальное значение engineDisplacement  в test:", test.engineDisplacement.max())
# Посмотрим на минимальное значения признака в test
print("Минимальное значение engineDisplacement  в test:", test.engineDisplacement.min())

In [ ]:
# Посмотрим на распределение выбросов
fig, ax = plt.subplots(figsize=(10, 5))
sns.histplot(data=data[(data.engineDisplacement > 4.8)], x='engineDisplacement', hue = 'train')
plt.title("Распределение выбросов engineDisplacement в датасете \n")
plt.show()

In [ ]:
# Посмотрим на распределение выбросов
fig, ax = plt.subplots(figsize=(10, 7))
sns.regplot(data=data[(data.engineDisplacement > 3.8) & (
    data.train == 1)], x="engineDisplacement", y="price")
plt.title("Зависимость price от engineDisplacement в train на выбросах по engineDisplacement \n")
plt.show()

In [ ]:
# Посмотрим на записи, которые привышают максимальный порог test
data[(data.engineDisplacement > 6.6)].describe()

Выбросы по признаку представлены как в train, так и в test части. Цена варьируется и увеличивается с увеличением объема двигателя (до определенного значения). Можно попробовать логарифмировать переменную. Также можно попробовать обработать выбросы заменой в строках > 6.6 литров на максимальное значение из теста.

In [ ]:
## Заменяем выбросы на максимальное значение признака в тесте
data.engineDisplacement = data.engineDisplacement.apply(lambda x: test.engineDisplacement.max(
) if x > test.engineDisplacement.max() else x)

#### description_words_count

In [ ]:
# Посмотрим на распределение выбросов
fig, ax = plt.subplots(figsize=(10, 5))
sns.histplot(data=data[(data.description_words_count > 307)], x='description_words_count', hue = 'train')
plt.title("Распределение выбросов description_words_count в датасете \n")
plt.show()

In [ ]:
# Посмотрим на распределение выбросов
fig, ax = plt.subplots(figsize=(10, 7))
sns.regplot(data=data[(data.description_words_count > 307) & (
    data.train == 1)], x="description_words_count", y="price")
plt.title("Зависимость price от engineDisplacement в train на выбросах по engineDisplacement \n")
plt.show()

Выбросы по признаку представлены как в train, так и в test части.
Цена варьируется и увеличивается с увеличением признака, поэтому пока оставим без изменений.
Можно попробовать логарифмировать переменную.

#### price

Проанализировать, но вообще log значение выбросы не содержит, а мы его используем в моделях.

In [ ]:
# Посмотрим на распределение выбросов
fig, ax = plt.subplots(figsize=(10, 5))
sns.histplot(data=data[(data.price > 2287500)&(data.train == 1)], x='price')
plt.title("Распределение выбросов price в датасете \n")
plt.show()

In [ ]:
data[(data.price > 2287500)&(data.train == 1)].describe(include=object)

In [ ]:
# Посмотрим, какие машины имеют особо высокие цены
data[(data.price > 2287500)&(data.train == 1)].groupby(['brand', 'model'])['price'].agg(['count', 'max'])
# Добавим список моделей в список
models_list = data[(data.price > 2287500)&(data.train == 1)].model.values

In [ ]:
# Вспомним, сколько авто таких моделей присутствует в тесте
data[(data.model.isin(models_list)) & (data.train == 0)].groupby(
    ['brand', 'model'])['model'].count()

Вывод: есть очень дорогие авто, которые представлены и в тестовой выборке.
Удалять строки нельзя, т.к. необходимо строить предсказания по авто таких моделей.

### Логарифмирование числовых признаков

Поскольку многие числовые переменные имеют смещенное распределение влево/вправо попробуем логарифмировать часть признаков.

In [ ]:
cols_to_log = ['enginePower', 'mileage', 'engineDisplacement', 'description_words_count']
# Применим логарифмирование ко всем числовым признакам
for col in cols_to_log:
    data[col] = data[col].apply(lambda w: np.log(w+1))

In [ ]:
# Посмотрим, как изменилось распределение
print("Диаграмы распределения числовых признаков после логарифмирования.")
fig, axes = plt.subplots(len(cols_to_log), figsize=(10, 15))
axes = axes.flatten()
i = 0

for col in cols_to_log:
    sns.distplot(data[col], ax=axes[i])
    plt.title(col)
    i = i + 1

Вывод: логарифмирование позволило привести признаки к более нормальному распределению. Оставим в таком виде.

### Отбор признаков для моделирования
Корреляционный анализ числовых признаков

In [ ]:
# Построим матрицу корреляций
plt.figure(figsize=(10, 6))
heatmap = sns.heatmap(data[data['train'] == 1][num_cols +
                                               target_col].corr(), vmin=-1, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('Матрица корреляций числовых и целевой переменных')
plt.show()

Сформируем список признаков, которые коллинеарны. Для этого выставим критерий наличия корреляции больше 0.8 или -0.8.

In [ ]:
# Сформируем сет со скоррелированными признаками
correlated_features = set()

# Удаляем целевые и служебные переменные из матрицы корреляций, т.к. корреляция с ней хорошо для модели
correlation_matrix = data[data['train'] == 1][num_cols].corr()

for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > 0.8:
            colname = correlation_matrix.columns[j]
            correlated_features.add(colname)

print('Список скоррелированных признаков на удаление из обучения модели:',
      correlated_features)

Вывод: найдены сильно скоррелированные признаки, но мы это уже видели во время анализа ранее и добавили признаки в список на исключение.

#### Удаление признаков перед моделированием

In [ ]:
# Удаляем признаки из датасета, которые решили удалить по ходу анализа
data.drop(cols_removal, axis=1, inplace=True)

In [ ]:
# Смотрим, какие признаки остались
print("После обработки остались следующие признаки:", data.columns)

#### Кодирование категориальных признаков

Основная логика выбора способа кодирования:

если у признака есть какая-то зависимость от порядкового номера категории или какая-то количественно выраженная разница между категориями, то используем label encoding.
если такого свойства нет и в списке большое количество значений, то пробуем one-hot-encoding.
После проведения экспериментов было решено применить label encoding ко всем категориальным признакам, т.к. даже после удаления статистически незначимых признаков, результаты были хуже.

#### Label Encoding

In [ ]:
# Labels encoding for all
cols_to_encode = list(set(data.columns) & set(cat_cols))
for colum in cols_to_encode:
    data[colum] = data[colum].astype('category').cat.codes

#### Поиск статистически значимых различий с помощью теста Стьюдента

Проверим, есть ли статистическая разница в распределении цен по категориальным и бинарным признакам, с помощью теста Стьюдента. Проверим нулевую гипотезу о том, что распределения цен по различным параметрам неразличимы.

In [ ]:
# Создадим список с колонками, которые удалим из данных ввиду их статистической незначимости
cat_cols_remove = []

# Проходим по колонкам, которые исключали из корреляционного анализа
for column in (list(set(data.columns).difference(num_cols+service_cols+target_col))):
    cat_cols_remove.append(get_stat_dif(column))

Вывод: присутсвуют признаки, для которых, не все значения категорий влияют на целевую переменную, такие признаки можно было бы удалить, если бы использовали one hot encoding, пока оставляем.

In [ ]:
#Оценим значимость числовых признаков
fig, ax = plt.subplots(figsize=(6, 3))
anova_df = data[data['train'] == 1].dropna().copy()
imp_num = pd.Series(f_regression(anova_df[list(set(data.columns) & set(num_cols))], anova_df['price_log'])[
                    0], index=list(set(data.columns) & set(num_cols)))
imp_num.sort_values(inplace=True)
imp_num.plot(
    kind='barh', title='Значимость непрерывных переменных по ANOVA F test по всем маркам')
plt.show()

In [ ]:
# Оценим значимость бинарных и категориальных признаков
fig, ax = plt.subplots(figsize=(15, 7))

anova_df = data[data['train'] == 1].dropna().copy()

# Labels encoding
cols_to_encode = list(set(anova_df.columns) & set(cat_cols))
for colum in cols_to_encode:
    anova_df[colum] = anova_df[colum].astype('category').cat.codes

imp_cat = pd.Series(mutual_info_regression(
    anova_df[list(set(data.columns) & set(bin_cols+cat_cols))], anova_df['price'], discrete_features=True), index=list(set(data.columns) & set(bin_cols+cat_cols)))
imp_cat.sort_values(inplace=True)
imp_cat.plot(kind='barh', title='Значимость категориальных переменных')
plt.show()

Выводы:

Из числовых признаков очень сильное влияние имеет productionDate, mileage, enginePower.
Из категориальных и бинарных самое сильное влияние оказывает model, brand, ownersCount.

### Стандартизация числовых признаков

In [ ]:
## Стандартизация числовых переменных
cols_to_scale = list(set(data.columns) & set(num_cols))
data[cols_to_scale] = StandardScaler().fit_transform(data[cols_to_scale].values)

In [ ]:
# Выделим тестовую и тренировочную части
X = data.query('train == 1').drop(
    ['train', 'price', 'price_log'], axis=1)
X_sub = data.query('train == 0').drop(
    ['train', 'price', 'price_log'], axis=1)

In [ ]:
# В качестве y указываем цену
y = data[data.train == 1]['price']

In [ ]:
#### Train Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

### Создадим "наивную" модель
Эта модель будет предсказывать среднюю цену по модели двигателя (mileage). C ней будем сравнивать другие модели.

In [ ]:
tmp_train = X_train.copy()
tmp_train['price'] = y_train

In [ ]:
# Находим median по экземплярам mileage в train и размечаем test
predict = X_test['mileage'].map(
    tmp_train.groupby('mileage')['price'].median())

# оцениваем точность
print(
    f"Точность наивной модели по метрике MAPE: {(mape(y_test, predict.values))*100:0.2f}%")

### Простая модель линейной регрессии

In [ ]:
from sklearn.linear_model import LinearRegression
linear_regr = LinearRegression().fit(X_train, np.log(y_train+1))
predict_test = np.exp(linear_regr.predict(X_test))
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

### CatBoost

In [ ]:
# model = CatBoostRegressor(iterations = 5000,
#                           random_seed = RANDOM_SEED,
#                           eval_metric='MAPE',
#                           custom_metric=['R2', 'MAE'],
#                           silent=True,
#                          )
# model.fit(X_train, y_train,
#          #cat_features=cat_features_ids,
#          eval_set=(X_test, y_test),
#          verbose_eval=0,
#          use_best_model=True,
#          #plot=True
#          )

# model.save_model('catboost_single_model_baseline.model')

In [ ]:
# # оцениваем точность
# predict = model.predict(X_test)
# print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.2f}%")

# # Точность модели по метрике MAPE: 15.03%

In [ ]:
# # Log Target
# # Попробуем взять таргет в логорифм - это позволит уменьшить влияние выбросов на обучение модели.
# model = CatBoostRegressor(iterations=5000,
#                           random_seed=RANDOM_SEED,
#                           eval_metric='MAPE',
#                           custom_metric=['R2', 'MAE'],
#                           silent=True,
#                           )
# model.fit(X_train, np.log(y_train),
#           # cat_features=cat_features_ids,
#           eval_set=(X_test, np.log(y_test)),
#           verbose_eval=0,
#           use_best_model=True,
#           # plot=True
#           )

# model.save_model('catboost_single_model_2_baseline.model')

# predict_test = np.exp(model.predict(X_test))
# predict_submission = np.exp(model.predict(X_sub))

# print(
#     f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

# # Точность модели по метрике MAPE: 12.36%

### xgboost

In [ ]:
xb = xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.5, learning_rate=0.03, \
                      max_depth=12, alpha=1, n_jobs=-1, n_estimators=1000)
xb.fit(X_train, np.log(y_train+1))

predict_test = np.exp(xb.predict(X_test))
predict_submission = np.exp(xb.predict(X_sub))

print(
      f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

# Точность модели по метрике MAPE: 12.01%

### GradientBoosting с подбором параметров

In [ ]:
# regularise(X_train, y_train)

In [ ]:
# gb = GradientBoostingRegressor(min_samples_split=2, learning_rate=0.03, max_depth=5, n_estimators=1000)
# gb.fit(X_train, np.log(y_train+1))

# predict_test = np.exp(gb.predict(X_test))
# predict_submission = np.exp(gb.predict(X_sub))

# print(
#       f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

# # Точность модели по метрике MAPE: 13.91%

### Stacking

In [ ]:
# scaler = StandardScaler() 
# X_train = scaler.fit_transform(X_train) 
# X_test = scaler.transform(X_test) 

# y_train = y_train 
# y_test = y_test

# cv = KFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)

# X_train = np.where(np.isnan(X_train), 0, X_train)
# X_test = np.where(np.isnan(X_test), 0, X_test)
# y_train = np.where(np.isnan(y_train), 0, y_train)

In [ ]:
# regr = RandomForestRegressor(n_estimators=300, min_samples_split=2, min_samples_leaf=1, 
#                              max_features=3, max_depth=19, bootstrap=True, random_state=RANDOM_SEED)

# stacked_features_train, stacked_features_test = generate_meta_features([
#                             regr,
#                             GradientBoostingRegressor(min_samples_split=2, learning_rate=0.03, max_depth=10, n_estimators=300),
#                             KNeighborsRegressor(n_neighbors=2, algorithm = 'ball_tree', weights = 'distance', p=1),
#                             RandomForestRegressor(random_state = RANDOM_SEED, n_jobs = -1, verbose = 1, max_depth=5, n_estimators=200),
#                             ExtraTreesRegressor(random_state=RANDOM_SEED), 
#                             RandomForestRegressor(random_state=RANDOM_SEED, max_depth=15) \
# ], X_train, X_test, y_train, cv)

In [ ]:
# egr.fit(X_train, y_train)

# predict_test = regr.predict(X_test)
# predict_submission = regr.predict(X_sub)

# print(
#       f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

# # Точность модели по метрике MAPE: 13.86%

### Выводы
CatBoost и GradientBoosting показывают хороший результат метрики, но лучший результат показал xgboost: MAPE 12.01.
Заметно улучшается результат после логарифмирования целевой переменной.

### SUBMISSION

In [ ]:
# Для финального сабмишена выбран np.exp(xb.predict(X_sub))
predict_submission = np.exp(xb.predict(X_sub))

sample_submission['price'] = np.round(np.array(predict_submission)/1000,2)*1000
sample_submission.to_csv(f'submission_v{VERSION}.csv', index=False)
sample_submission.head(10)